# ___Les variables et la mutabilité___

---

## La notion de mutabilité

---

- Tentons d'abord quelque chose:

In [23]:
let a = 42;
dbg!(a);
a = 84;
dbg!(a);

Error: cannot assign twice to immutable variable `a`

**J'assigne une valeur 42 à ma variable `a`, je fais afficher cette valeur, j'assigne une nouvelle valeur 84 puis je veux afficher cette nouvelle valeur. Pourquoi ça ne compile donc pas ?**  

Le compilateur l'explique, ma variable `a` n'est pas mutable. En Rust je suis obligé de spécifier qu'une variable sera mutable, c'est une décision de conception qui vise à encourager un style de programmation qui favorise l'immutabilité autant que possible. **Cela permet de rendre plus facile la détection des erreurs de concurrence et des problèmes de sécurité.**

* **Il faut fonc ajouter le mot clef `mut`.**

In [24]:
let mut a = 42; // Je rajoute ici le mot-clef mut
dbg!(a + 4);
a = 84;
dbg!(a)

[src/lib.rs:119] a + 4 = 46
[src/lib.rs:137] a = 84


84

*Par défaut, le compilateur génère ces warnings en cas de non-utilisation de re assignation d'une variable mutable.*

In [25]:
let mut z = 42;
dbg!(z);

[src/lib.rs:121] z = 42


![WARN](pictures/mut-warning.png)

## Les constantes

---

*Les constantes introduites par le mot-clef `const` se différencient des variables non-mutables classiques déclarées par `let` sans `mut` par:*

> - le fait que déjà une variable dite constante avec `const` ne pourra jamais être mutable, en termes de design de code, cela peut jouer.
> - En interne, des substitutions peuvent s'operer, ce qui peut permettre certaines optimisations.
> - Enfin la déclaration d'une constante touche tout le **scope** dans laquelle elle est déclarée et ses sous-éléments, et cela, indépendamment de l'endroit ou elle est déclarée. On peut ainsi faire des globales grâces au mot-clef const, dans ce cas, par convention, le nom de la constante sera mis en majuscule.

In [2]:
const GLOBALE: usize = 6; // Ceci est une globale, mise en majuscule

fn main() {
    const r: usize = 42;
    fn test() {
        dbg!(r); // r est declare dans le scope parent
        dbg!(s); // s est declare dans le scope parent
        dbg!(GLOBALE); // On accede a la variable globale
    }
    test();
    const s: usize = 12; // Declaration apres l'appel de la fonction test()
    dbg!(s);
}
main()

[src/lib.rs:7] r = 42
[src/lib.rs:8] s = 12
[src/lib.rs:9] GLOBALE = 6
[src/lib.rs:13] s = 12


()

- Il aurait été impossible d'utiliser r dans la fonction test() si la variable n'avait pas été instanciée par `const`.

In [27]:
fn main() {
    let new_var: usize = 42;
    fn test() {
        dbg!(new_var);
    }
}
main()

Error: can't capture dynamic environment in a fn item

- une `const` est accessible dans son scope et celui de ses enfants, mais seulement dans ces derniers.

In [28]:
fn main() {
    const j: usize = 42;
}
fn aux() {
    dbg!(j);
}

Error: cannot find value `j` in this scope

*Elles ne seront pas abordées dans ce cours, mais rust implémente petit à petit des fonctions constantes dites `const fn`, ce sont des fonctions qui sont évaluées a la compilation ce qui peut améliorer les performances du programme.*

## La redéfinition d'une variable et le shadowing

---

- Il existe en Rust un principe appelé **shadowing** qui permet de réutiliser le même nom pour une nouvelle variable, l'ancienne valeur sera oubliée.

In [29]:
let x: u32 = 12;
dbg!(x);
let x: i64 = -1;
dbg!(x);

[src/lib.rs:129] x = 12
[src/lib.rs:131] x = -1


In [30]:
fn main() {
    let x = 5;
    let x = x + 1; // Ici, c'est deja un nouveau x
    {
        let x = x * 2; // Cette nouvelle variable x remplace l'ancienne UNIQUENENT dans ce scope
        println!("The value of x in the inner scope is: {x}");
        let x = 1_123_999_123;
    }
    println!("The value of x is: {x}"); // On retrouve l'ancienne valeur de x
}
main()

The value of x in the inner scope is: 12
The value of x is: 6


()

*Un let ne peut pas `shadow` une constante.*

In [31]:
const G: usize = 64;

fn ma_fonction() {
    let G: u64 = 32;
    dbg!(G);
}
ma_fonction()

Error: mismatched types

## Les variables statiques

---

- Il existe un autre moyen en Rust de créer des variables globales : plutôt que d'utiliser le mot-clef **const** qui ne désigne que des variables globales constantes, on peut utiliser a la place le mot clef **static**. Ces statics peuvent être mutables, ainsi contrairement aux const, les static sont contenues dans un emplacement en mémoire, ce n'est pas une simple substitution des valeurs a la compilation. Les static existent pendant toute la durée de l'exécution du programme et ne sont pas détruites à la fin.

In [32]:
static VALUE: usize = 42;

dbg!(VALUE);

[src/lib.rs:129] VALUE = 42


- Comme on l'a vu precedement, on utilisera le mot-clef **mut** pour pouvoir les rendre modifiables. Seulement, un petit problème va se poser, cherchons à comprendre pourquoi en testant :

In [33]:
static mut VALUE_MUTABLE: usize = 84;

dbg!(VALUE_MUTABLE);

Error: unused variable: `x`

Error: use of mutable static is unsafe and requires unsafe function or block

Error: use of mutable static is unsafe and requires unsafe function or block

Visiblement, le compilateur n'est pas d'accord et nous indique que l'on doit impérativement utiliser le mot-clef unsafe, mais pourquoi donc ?!? En fait, cela vient du fait que le compilateur considère que nous pouvons avoir plusieurs threads qui cherchent a accéder ou a modifier notre variable en même temps ce qui provoquerait une **race condition**. Il nous invite ainsi à designer le code comme unsafe, une façon de demander au programmeur d'assumer son choix, ainsi s'il se passe un problème durant l'exécution du code, nous nous efforcerons de chercher en premier les causes dans les blocs dit "unsafe".

In [34]:
unsafe {
    dbg!(VALUE_MUTABLE);
}

[src/lib.rs:146] VALUE_MUTABLE = 42


()

Ou bien si l'on désire ne pas faire du code unsafe, on peut utiliser un Mutex:

In [35]:
use std::sync::Mutex;

static VAR_MUT_PRETECTED: Mutex<u32> = Mutex::new(42);

dbg!(*VAR_MUT_PRETECTED.lock().unwrap());


[src/lib.rs:129] *VAR_MUT_PRETECTED.lock().unwrap() = 42


*Nous verrons plus tard lors du chapitre Pointeurs et Références ce que fait la petite étoile et le trait Deref...

Notons enfin qu'il n'est pas possible d'utiliser le *shadowing* sur les variables statiques.

In [36]:
static C: usize = 42;

fn test() {
    let C: f64 = std::f64::consts::PI;
}

Error: let bindings cannot shadow statics

## Exercices

---

### Quelle est l'erreur dans le code ci-dessous ?
```
fn main() {
    let x = 5;
    dbg!(x);
    x = 10;
    println!("x = {}", x);
}
```

**On tente de modifier la valeur de x alors que x n'est pas mutable !**

In [37]:
fn main() {
    let x = 5;
    dbg!(x);
    x = 10;
    println!("x = {}", x);
}

Error: cannot assign twice to immutable variable `x`

### Est-ce que ce code compile ? Pourquoi ?
```
fn main() {
    let mut x: usize = 43;
    dbg!(x);
    let x: f64 = 3.14;
    dbg!(x);
}
```

**Oui, il compile, c'est une utilisation triviale du shadowing.**

### Est-ce que ce code compile ? Pourquoi ?
```
fn main() {
    let mut x: usize = 43;
    dbg!(x);
    const x: f64 = 3.14;
    dbg!(x);
}
```

**La const etant vivante dans tout le bloc d'instruction, le let tente de l'ecraser, c'est interdit.**

In [38]:
fn main() {
    let mut x: usize = 43;
    dbg!(x);
    const x: f64 = 3.14;
    dbg!(x);
}

Error: let bindings cannot shadow constants

In [39]:
fn main() {
    let mut x: usize = 43;
    dbg!(x);
    let x: f64 = 3.14;
    dbg!(x);
}
main()

[src/lib.rs:15] x = 43
[src/lib.rs:17] x = 3.14


()

### Corriger le code suivant:

In [3]:
static mut END: usize = 23;

fn main() {
    dbg!(END);
    END = END * 2;
    dbg!(END);
}
main()

Error: use of mutable static is unsafe and requires unsafe function or block

Error: use of mutable static is unsafe and requires unsafe function or block

Error: use of mutable static is unsafe and requires unsafe function or block

Error: use of mutable static is unsafe and requires unsafe function or block

Error: use of mutable static is unsafe and requires unsafe function or block

Error: use of mutable static is unsafe and requires unsafe function or block

In [41]:
static mut END: usize = 23;

fn main() {
    unsafe {
        dbg!(END);
        END = END * 2;
        dbg!(END);
    }
}
main()

[src/lib.rs:15] END = 23
[src/lib.rs:17] END = 46


()